## How to write a `Task`
---

**TL;DR If you take one thing away from this talk, it is
go to http://pipelines.lsst.io, then click on [lsst.pipe.base](https://pipelines.lsst.io/modules/lsst.pipe.base/index.html).** 


---
On the landing page for `lsst.pipe.base` documenation https://pipelines.lsst.io/modules/lsst.pipe.base/index.html, you'll see a number of tutorials on how to use `Tasks` and how to create one. 



`CmdlineTask` extends `Task` with commandline driver utils for use with Gen2 Butlers, and will be deprecated soon. However, not all the links under "CommandlineTask" will become obsolete. For example, [Retargeting subtasks of command-line tasks](https://pipelines.lsst.io/modules/lsst.pipe.base/command-line-task-retargeting-howto.html) will live on.

In this talk, we'll only quickly go over the prerequisites for understanding the `PipelineTask`:

1. https://pipelines.lsst.io/modules/lsst.pipe.base/task-framework-overview.html
2. https://pipelines.lsst.io/modules/lsst.pipe.base/creating-a-task.html

For more background info, I highly recommend you look at Russell's 2015 Bootcamp treatment of `Tasks`: [pptx slides](https://github.com/lsst-dm/Oct15_bootcamp/blob/master/documents/writing%20a%20task.pptx?raw=true) and [video](https://vimeo.com/album/3606100/video/142210167)

---

### What is a Task?
Tasks implement astronomical data processing functionality. They are:
* **Configurable**: Modify a task’s behavior by changing its configuration. Automatically apply camera-specific modifications
* **Hierarchical**: Tasks can call other tasks as subtasks
* **Extensible:** Replace (“retarget”) any subtask with a variant. Write your own subclass of a task.


### Using a Task:

In [1]:
# Edited highlights of ${PIPE_TASKS_DIR}/example/exampleStatsTask.py
import sys
import numpy as np
from lsst.geom import Box2I, Point2I, Extent2I
from lsst.afw.image import MaskedImageF
from lsst.pipe.tasks.exampleStatsTasks import ExampleSimpleStatsTask, ExampleSigmaClippedStatsTask

In [2]:
# Load a MaskedImageF -- an image containing floats
# together with a mask and a per-pixel variance.

WIDTH = 40
HEIGHT = 20

maskedImage = MaskedImageF(Box2I(Point2I(10, 20),
                                 Extent2I(WIDTH, HEIGHT)))
x = np.random.normal(10, 20, size=WIDTH*HEIGHT)

# Because we are shoving it into an ImageF and numpy defaults
# to double precision
X = x.reshape(HEIGHT, WIDTH).astype(np.float32)  
im = maskedImage.image
im.array = X

# We initialize the Task once but can call it many times.
task = ExampleSimpleStatsTask()

# Simply call the .run() method with the MaskedImageF.
result = task.run(maskedImage)

# And print the result.
print(result)

Struct(mean=11.37018704397138; meanErr=0.6982692625096175; stdDev=19.7500372245872; stdDevErr=0.49344223978699775)


## Using a Task with configuration

In [3]:
# Edited highlights of ${PIPE_TASKS_DIR}/example/exampleStatsTask.py

config1 = ExampleSigmaClippedStatsTask.ConfigClass(numSigmaClip=1)

config2 = ExampleSigmaClippedStatsTask.ConfigClass()
config2.numSigmaClip = 3

task1 = ExampleSigmaClippedStatsTask(config=config1)
task2 = ExampleSigmaClippedStatsTask(config=config2)

print(task1.run(maskedImage).mean)
print(task2.run(maskedImage).mean)

11.08069926686585
11.37018704397138


In [4]:
# Configs are validated. Should Raise:
# config3 = ExampleSigmaClippedStatsTask.ConfigClass(numSigmaClip="ten")